In [ ]:
!pip install ultralytics
!pip install deep-sort-realtime dotenv anthropic
!pip install azure-cognitiveservices-vision-computervision
!pip install --upgrade azure-cognitiveservices-vision-computervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from dotenv import load_dotenv
load_dotenv("/content/API Key.env")
load_dotenv(dotenv_path="/content/API Key.env", override=True)

True

In [ ]:
!nvidia-smi

Tue Apr  8 13:30:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             45W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
from ultralytics import YOLO
model = YOLO("/content/results_version_3/content/runs/detect/train2/weights/best.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
results[0].names

{0: 'car',
 1: 'cross_walk',
 2: 'green_lights',
 3: 'lisence_plate',
 4: 'motorcycle',
 5: 'nothing',
 6: 'red_lights'}

# **Extract lisence plate**

In [ ]:
import time
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
from PIL import Image
from google import genai
from dotenv import load_dotenv
import os
from google.genai import types
import pathlib
import PIL.Image
import requests
import anthropic


class ExtractLicensePlates:
    def __init__(self, image):
        self.original_image = image
        self._init_azure_client()
        self._init_gemini_client()
        self._init_claude_client()
        self.image = self._ensure_image_path(image)

    def _init_azure_client(self):
        endpoint = "https://tamhuynh278.cognitiveservices.azure.com/"
        key = os.getenv("AZURE_OCR")
        self.azure_client = ComputerVisionClient(
            endpoint,
            CognitiveServicesCredentials(key)
        )

    def _init_gemini_client(self):
        from google import genai
        gemini_key = os.getenv("GEMINI_API_KEY")
        self.gemini_client = genai.Client(api_key = gemini_key)

    def _init_claude_client(self):
        claude_key = os.getenv("CLAUDE_API_KEY_TON")
        self.claude_client = anthropic.Anthropic(api_key=claude_key)

    def _ensure_image_path(self, image):
        if isinstance(image, str):
            return image

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced = clahe.apply(gray)
        output_path = os.path.abspath("new.jpg")
        cv2.imwrite(output_path, enhanced)

        return output_path

    def extract_text_with_plate_recogniation(self):
        try:
            with open(self.image, 'rb') as fp:
                response = requests.post(
                    'https://api.platerecognizer.com/v1/plate-reader/',
                    files=dict(upload=fp),
                    headers={'Authorization': f'Token {os.getenv("PLATES_RECOGNI")}'}
                )

                if response.status_code == 201:
                    result = response.json()
                    for res in result['results']:
                        return res['plate'].upper()
                else:
                    return response.status_code, response.text
        except Exception as e:
            return None


    def extract_text_with_azure(self):
        try:
            with open(self.image , "rb") as image_stream:
                read_response = self.azure_client.read_in_stream(image_stream, raw=True)
            operation_location = read_response.headers["Operation-Location"]
            operation_id = operation_location.split("/")[-1]

            while True:
                result = self.azure_client.get_read_result(operation_id)
                if result.status.lower() not in ["notstarted", "running"]:
                    break
                time.sleep(1)

            if result.status == "succeeded":
                extracted_text = []
                if hasattr(result, 'analyze_result') and hasattr(result.analyze_result, 'read_results'):
                    for page in result.analyze_result.read_results:
                        for line in page.lines:
                            extracted_text.append(line.text)
                    return "\n".join(extracted_text) if extracted_text else None
            return None
        except Exception as e:
            return None

    def extract_text_with_claude(self):
        try:
            system_prompt = """
            You are an OCR engine specialized in reading license plates from images.

            Your task:
            - Extract the license plate only.
            - Return only the license plate as a single line of plain text.
            - Do not include any explanations, apologies, or additional messages.
            - If the license plate cannot be identified, return only: "unknown"
            - Output should never include phrases like "The extracted license plate is..." or "Unable to extract...
            """

            message_list = [
                {
                    "role": 'user',
                    "content": [
                        {"type": "image", "source": {"type": "base64", "media_type": "image/jpeg", "data": get_base64_encoded_image(self.image)}},
                        {"type": "text", "text": system_prompt}
                    ]
                }
            ]

            response = self.claude_client.messages.create(
                model="claude-3-haiku-20240307",
                max_tokens=600,
                messages=message_list
                )
            return response.content[0].text

        except Exception as e:
            return None

    def extract_text_with_gemini(self):
        from google import genai

        try:
            system_prompt = """
            You are an OCR engine specialized in reading license plates from images.

            Your task:
            - Extract the license plate only.
            - Return only the license plate as a single line of plain text.
            - Do not include any explanations, apologies, or additional messages.
            - If the license plate cannot be identified, return only: "unknown"
            - Output should never include phrases like "The extracted license plate is..." or "Unable to extract..."
            """
            b64_image = types.Part.from_bytes(
                data=pathlib.Path(self.image).read_bytes(),
                mime_type="image/jpeg"
            )
            response = self.gemini_client.models.generate_content(
                model="gemini-1.5-pro",
                contents=[system_prompt, b64_image ])
            return response.text
        except Exception as e:
            return None

    def analyze_plates(self, text_list):
        clean_text = []
        for text in text_list:
            if isinstance(text, str):
                clean_text.append(text)
            elif isinstance(text, list):
                clean_text.extend([str(item) for item in text])
            elif isinstance(text, tuple):
                continue
            elif isinstance(i, dict):
                continue
            else:
                clean_text.append(str(text))

        clean_text = "\n".join(clean_text)
        try:
            system_prompt = """
                You are a helpful AI assistant.
                - Extract the **best matching license plate**, even if it is **incomplete**.
                - Prioritize strings that follow **partial or full plate formats**, such as:
                - Motorcycles: XXXX XXXXX (examples: 43E1 16480, 72A2 02501, ....)
                - Cars: XXX XXXXX (examples: 29A 12345, 50C 75820, ....)
                - Please remember to put a space for me

                Instructions:
                - Return **only** the most likely license plate string.
                - Return only the license plate as a single line of plain text.
                - Do not include any explanations, apologies, or additional messages.
                """

            response = self.claude_client.messages.create(
                model="claude-3-haiku-20240307",
                max_tokens=100,
                system=system_prompt,
                messages=[{"role": "user", "content": clean_text}]
            )
            return response.content[0].text
        except Exception as e:
            return None


    def run_method_OCR(self):
        extracted_text = self.extract_text_with_plate_recogniation()
        if extracted_text:
            return extracted_text

        extracted_text = self.extract_text_with_claude()
        if extracted_text:
            return extracted_text

        extracted_text = self.extract_text_with_azure()
        if extracted_text:
            return extracted_text

        extracted_text = self.extract_text_with_gemini()
        if extracted_text:
            return extracted_text

        if not extracted_text:
            return "UNKNOWN"


# **Logic handle and draw stopline**

In [ ]:
import math

def is_vehicle_crossed_stop_line(vehicle_bbox, stop_line):
    _, _, x2, y2 = vehicle_bbox
    return y2 >= stop_line["y"] and stop_line["x_min"] <= x2 <= stop_line["x_max"]


def create_stop_line_from_crosswalk(crosswalk_bbox, offset=10, min_length=50):
    x1, y1, x2, y2 = crosswalk_bbox
    stop_line_y = y1 - offset

    length = x2 - x1
    if length < min_length:
        center = (x1 + x2)/2
        x1 = center - min_length/2
        x2 = center + min_length/2
    return {"y": stop_line_y, "x_min": int(x1), "x_max": int(x2)}



def draw_stop_line(frame, stop_line, thickness=2):
    cv2.line(frame,
             (int(stop_line["x_min"]), int(stop_line["y"])),
             (int(stop_line["x_max"]), int(stop_line["y"])),
             (0, 0, 255),
             thickness)

In [ ]:
def is_valid_plate_flexible(plate_bbox, vehicle_bbox,
                          min_iou_ratio=0.2,
                          max_edge_margin=20,
                          bottom_height_ratio=0.7):

    inter_x1 = max(plate_bbox[0], vehicle_bbox[0])
    inter_y1 = max(plate_bbox[1], vehicle_bbox[1])
    inter_x2 = min(plate_bbox[2], vehicle_bbox[2])
    inter_y2 = min(plate_bbox[3], vehicle_bbox[3])

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    plate_area = (plate_bbox[2] - plate_bbox[0]) * (plate_bbox[3] - plate_bbox[1])
    iou = inter_area / plate_area if plate_area > 0 else 0

    left_close = abs(plate_bbox[0] - vehicle_bbox[0]) <= max_edge_margin
    right_close = abs(plate_bbox[2] - vehicle_bbox[2]) <= max_edge_margin

    plate_center_y = (plate_bbox[1] + plate_bbox[3]) / 2
    vehicle_bottom = vehicle_bbox[1] + (vehicle_bbox[3] - vehicle_bbox[1]) * bottom_height_ratio
    is_at_bottom = plate_center_y >= vehicle_bottom

    return (iou >= min_iou_ratio or left_close or right_close) and is_at_bottom

# **Pipeline object detections**

In [ ]:
class ObjectDetector:
    def __init__(self, model):
        self.model = model
        self.red_lights = []
        self.motorcycles = []
        self.cars = []
        self.vehicles = []
        self.cross_walks = []
        self.lisence_plates = []
        self.dets = []

    def detect_objects(self, image):
        results = self.model(image)
        detections = results[0].boxes.data.cpu().numpy()

        CONF_THRESHOLDS = {0: 0.4, 1: 0.3, 3: 0.6, 4: 0.4}

        for det in detections:
            x1, y1, x2, y2, conf, cls = det
            cls = int(cls)

            if cls in CONF_THRESHOLDS and conf < CONF_THRESHOLDS[cls]:
                continue

            if cls in [0, 4]:
                width = x2 - x1
                height = y2 - y1
                self.vehicles.append(([x1, y1, width, height], conf, cls))

            if cls in [3, 6]:
                width = x2 - x1
                height = y2 - y1
                self.dets.append(([x1, y1, width, height], conf, cls))

            if cls == 3:
                self.lisence_plates.append([x1, y1, x2, y2])
            elif cls == 0:
                self.motorcycles.append([x1, y1, x2, y2])
            elif cls == 4:
                self.motorcycles.append([x1, y1, x2, y2])
            elif cls == 1:
                self.cross_walks.append([x1, y1, x2, y2])
            elif cls == 6:
                self.red_lights.append([x1, y1, x2, y2])

In [ ]:
class TrackerObject:
    def __init__(self, tracker_vehicles, tracker, frame):
        self.tracker_vehicles = tracker_vehicles
        self.tracker = tracker
        self.frame = frame

    def deep_sort_vehicle(self, vehicle):
        tracks_vehicles = self.tracker_vehicles.update_tracks(vehicle, frame=self.frame)
        return tracks_vehicles

    def deep_sort_dets(self, dets):
        tracks = self.tracker.update_tracks(dets, frame=self.frame)
        return tracks

In [ ]:
class HandleTrackVehicles:
    def __init__(self, track_vehicles, frame):
        self.frame = frame
        self.vehicle_info = []
        self.track_vehicles = track_vehicles

        self.track_id = None
        self.bbox = None

    def handle_tracks_vehicle(self):
        for track_v in self.track_vehicles:
            if not track_v.is_confirmed() or track_v.time_since_update > 1:
                continue
            self.track_id = track_v.track_id
            self.bbox = track_v.to_tlbr()
            x1_V, y1_V, x2_V, y2_V = map(int, self.bbox)

            self.vehicle_info.append({
                'track_id': self.track_id,
                'bbox': self.bbox
            })

            Color_Pala(self.frame).draw_color(self.track_id, self.bbox)

In [ ]:
class Color_Pala():
    color_palette = defaultdict(lambda: (
        random.randint(0, 255),
        random.randint(0, 255),
        random.randint(0, 255)
    ))

    def __init__(self, frame):
        self.frame = frame

    def draw_color(self, track_id, bounding_box):
        x1, y1, x2, y2 = map(int, bounding_box)
        color = Color_Pala.color_palette[track_id]
        label = f"ID: {track_id}"
        (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)
        cv2.rectangle(self.frame,
                (x1, y1),
                (x2, y2),
                color, 2)
        cv2.putText(self.frame, label,
                    (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

In [ ]:
class HandleTracks:
    def __init__(self, tracks, vehicle_info, cars, motorcycles,
                 license_plates, red_lights, stop_line, frame, vehicle_to_license):
        self.tracks = tracks
        self.vehicle_info = vehicle_info
        self.cars = cars
        self.motorcycles = motorcycles
        self.license_plates = license_plates
        self.red_lights = red_lights
        self.stop_line = stop_line
        self.frame = frame

    def handle_tracks(self):
        for track in self.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            track_id_plate = track.track_id
            bbox_plate = track.to_tlbr()
            x1_L, y1_L, x2_L, y2_L = map(int, bbox_plate)

            self.handle_vehicle_list(x1_L, y1_L, x2_L, y2_L)

            Color_Pala(self.frame).draw_color(track_id_plate, bbox_plate)

    def handle_vehicle_list(self, x1_L, y1_L, x2_L, y2_L):
        for vehicle in self.vehicle_info:
            track_id_vehicle = vehicle['track_id']
            bbox_vehicle = vehicle['bbox']
            x1_V_current, y1_V_current, x2_V_current, y2_V_current = map(int, vehicle['bbox'])

            if (len(self.cars) > 0 or len(self.motorcycles) > 0) and len(self.license_plates) > 0 and len(self.red_lights) > 0:
                if self.stop_line is not None:
                    if is_vehicle_crossed_stop_line([x1_V_current, y1_V_current, x2_V_current, y2_V_current], self.stop_line):
                        if is_valid_plate_flexible([x1_L, y1_L, x2_L, y2_L], [x1_V_current, y1_V_current, x2_V_current, y2_V_current]):
                            crop_img = self._get_cropped_plate(x1_L, y1_L, x2_L, y2_L)

                            if track_id_vehicle not in vehicle_to_license:
                                vehicle_to_license[track_id_vehicle] = []
                            vehicle_to_license[track_id_vehicle].append(crop_img)


    def _get_cropped_plate(self, x1_L, y1_L, x2_L, y2_L):
        center_x = (x1_L + x2_L) // 2
        center_y = (y1_L + y2_L) // 2

        x1_pad = max(center_x - 335 // 2, 0)
        y1_pad = max(center_y - 462 // 2, 0)
        x2_pad = x1_pad + 335
        y2_pad = y1_pad + 462

        x2_pad = min(x2_pad, frame.shape[1])
        y2_pad = min(y2_pad, frame.shape[0])
        x1_pad = max(x2_pad - 335, 0)
        y1_pad = max(y2_pad - 462, 0)

        crop_img = self.frame[y1_pad:y2_pad, x1_pad:x2_pad]

        return crop_img


In [ ]:
from google.colab.patches import cv2_imshow
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
from ultralytics import YOLO
import random
from collections import defaultdict
import time

DISTANCE_THRESHOLD = 10

CONFIRMATION_FRAMES = 3
tracker = DeepSort(
    max_age=10,
    nn_budget=50,
    n_init=CONFIRMATION_FRAMES,
)

tracker_vehicles = DeepSort(
    max_age=10,
    nn_budget=50,
    n_init=CONFIRMATION_FRAMES,
)


video_path = "/content/4Ly giai chuyen la o TP Vinh đen đo đuoc đi thang tai mot so giao (online-video-cutter.com).mp4"
cap = cv2.VideoCapture(video_path)

width, height, fpbs = (int(cap.get(x)) for x in
           (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('/content/output_video__version_3.avi', fourcc, fpbs,
            (width, height))

frame_count = 0

persistent_crosswalk = None
last_crosswalk_time = 0
crosswalk_timeout = 1.0

vehicle_to_license = {}


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count +=1

    OBJECT_DETECTOR = ObjectDetector(model)
    OBJECT_DETECTOR.detect_objects(frame)

    TRACKER_OBJECT = TrackerObject(tracker_vehicles, tracker, frame)
    TRACKERS_VEHICLES = TRACKER_OBJECT.deep_sort_vehicle(OBJECT_DETECTOR.vehicles)
    TRACKERS = TRACKER_OBJECT.deep_sort_dets(OBJECT_DETECTOR.dets)


    current_time = time.time()
    if len(OBJECT_DETECTOR.cross_walks) > 0:
        persistent_crosswalk = OBJECT_DETECTOR.cross_walks[0]
        last_crosswalk_time = current_time

    stop_line = None
    if persistent_crosswalk is not None and (current_time - last_crosswalk_time) < crosswalk_timeout:
        stop_line = create_stop_line_from_crosswalk(persistent_crosswalk)
        draw_stop_line(frame, stop_line)
    else:
        persistent_crosswalk = None


    height_frame, width_frame = frame.shape[:2]

    HANDLE_TRACK_VEHICLES = HandleTrackVehicles(TRACKERS_VEHICLES, frame)
    HANDLE_TRACK_VEHICLES.handle_tracks_vehicle()


    HANDLE_TRACKS = HandleTracks(TRACKERS, HANDLE_TRACK_VEHICLES.vehicle_info, OBJECT_DETECTOR.cars,
                                OBJECT_DETECTOR.motorcycles, OBJECT_DETECTOR.lisence_plates,
                                OBJECT_DETECTOR.red_lights, stop_line, frame, vehicle_to_license)

    HANDLE_TRACKS.handle_tracks()


    for track_id, lisence_img in vehicle_to_license.items():
        text_fragments  = []
        extract_plates = None
        full_text = None

        for bbox in lisence_img:
            extract_plates = ExtractLicensePlates(bbox)
            plate_number = extract_plates.run_method_OCR()
            if plate_number:
                text_fragments.append(plate_number)
            time.sleep(1)

        if extract_plates is not None and text_fragments:
            full_text = extract_plates.analyze_plates(text_fragments)

        (text_width, text_height), _ = cv2.getTextSize(full_text.strip() if full_text else "UNKNOWN" ,
                                                    cv2.FONT_HERSHEY_SIMPLEX,
                                                    1, 2)
        cv2.putText(frame, full_text.strip() if full_text else "UNKNOWN" ,
                        (width_frame - text_width - 20, height_frame - 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)

        print(full_text)

    out.write(frame)

cap.release()
out.release()